In [42]:
import pandas as pd

df = pd.read_csv("scenarios_uncleaned.csv")
df['prompt'][0]

"<|im_start|>human1\nDude, do you ever think we're living in a simulation?\n<|im_end|>\n\n<|im_start|>human2"

In [ ]:
from scenarios_model import scenarios_json

# Create mapping
template_map = {s['name']: s['context'] for s in scenarios_json}

# Map and concatenate
df['full_prompt'] = df['category'].map(template_map) + "\n" + df['prompt']

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_name = "Qwen/Qwen2.5-14B-Instruct"
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map={"": "cuda:0"})
tokenizer = AutoTokenizer.from_pretrained(model_name, padding_side="left")

Loading checkpoint shards: 100%|██████████| 8/8 [00:06<00:00,  1.22it/s]


In [49]:
prompts = df['full_prompt'].tolist()

all_outputs = []
batch_size = len(df)

for i in range(0, len(prompts), batch_size):
    batch_prompts = prompts[i:i+batch_size]
    # decode all escape sequences like \n, \t, \; etc.
    batch_prompts = [bytes(p, "utf-8").decode("unicode_escape") for p in batch_prompts]
    
    inputs = tokenizer(batch_prompts, padding=True, truncation=True, return_tensors="pt").to("cuda")
    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=100, temperature=0.7)
    decoded = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    all_outputs.extend(decoded)

df['original_completion'] = all_outputs

In [61]:
df.to_csv("scenarios_cleaned.csv", index=False)